In [15]:
%load_ext autoreload
%autoreload 2
import os
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import sasc.config
import numpy as np
from collections import defaultdict
import pandas as pd
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR
# loo_pc = joblib.load(f'{FMRI_DIR}/voxel_neighbors_and_pcs/loo_pc_UTS02.pkl')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Story data
Load all the results from the stories into a single pickle file.

In [16]:

output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot_story_data.pkl")
story_mapping = {
    "default/uts02_pilot_gpt4_mar28___ver=v4_noun___seed=1": "GenStory1_resps.npy",
    "default/uts02_pilot_gpt4_mar28___ver=v4_noun___seed=3": "GenStory2_resps.npy",
    "default/uts02_pilot_gpt4_mar28___ver=v4_noun___seed=4": "GenStory3_resps.npy",
    "default/uts02_pilot_gpt4_mar28___ver=v5_noun___seed=1": "GenStory4_resps.npy",
    "default/uts02_pilot_gpt4_mar28___ver=v5_noun___seed=2": "GenStory5_resps.npy",
    "default/uts02_pilot_gpt4_mar28___ver=v5_noun___seed=4": "GenStory6_resps.npy",
}

In [17]:
story_names = story_mapping.keys()  # os.listdir(STORIES_DIR)
cluster_neighbors = joblib.load(
    join(FMRI_DIR, 'brain_tune', "voxel_neighbors_and_pcs",
         "cluster_neighbors_v1.pkl")
)

# process with timings
stories_data_dict = defaultdict(list)
for story_name in story_names:
    timings = pd.read_csv(
        join(STORIES_DIR, story_name, "timings_processed.csv"))
    # timings['time_running'] += 6  # -= 5  # += 6
    stories_data_dict["timing"].append(
        timings
    )
    stories_data_dict["story_name_original"].append(story_name.split('/')[1])
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    stories_data_dict["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    stories_data_dict["prompts"].append(
        open(join(STORIES_DIR, story_name, "prompts.txt"), "r")
        .read()
        .split("\n\n")
    )

    # rows
    story_metadata_per_paragraph = pd.read_csv(
        join(STORIES_DIR, story_name, "rows.csv"))
    story_metadata_per_paragraph["voxel_num"] = story_metadata_per_paragraph.apply(
        lambda row: convert_module_num_to_voxel_num(
            row["module_num"], row["subject"]),
        axis=1,
    )
    story_metadata_per_paragraph["stability_score"] = story_metadata_per_paragraph.apply(
        lambda row: add_stability_score(row["module_num"], row["subject"]),
        axis=1,
    )
    story_metadata_per_paragraph = story_metadata_per_paragraph[
        [
            "expl",
            "subject",
            "module_num",
            "roi_anat",
            "roi_func",
            "voxel_num",

            # metrics
            "stability_score",
            "top_explanation_init_strs",
            # "top_explanation_init_strs_llama",
            "fmri_test_corr",
            "top_score_synthetic",
            "top_score_normalized",
        ]
    ]
    story_metadata_per_paragraph["cluster_nums"] = story_metadata_per_paragraph["voxel_num"].map(
        cluster_neighbors)
    stories_data_dict["rows"].append(story_metadata_per_paragraph)


joblib.dump(stories_data_dict, output_file)

['/home/chansingh/automated-explanations/results/processed/pilot_story_data.pkl']

In [14]:
stories_data_dict = joblib.load(output_file)
stories_data_dict['rows'][0].to_csv('uts02_default_voxels.csv')